# voter data project
### by Cris Crawford
The purpose of this file is to unzip the voter activity file in my Google cloud bucket, convert the resulting data into parquet files, and put the parquet files back into the Google cloud bucket.

In [1]:
import zipfile
import pandas as pd
from google.cloud import storage

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/cris/.gc/keys.json'

In [3]:
google_application_credentials = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')
print(google_application_credentials)

/home/cris/.gc/keys.json


### The following cell was run once to unzip the data file.
It created 351 files, each holding the information about elections in a particular city or town. All other cells can be run repeatedly.

In [33]:
# Initialize GCS client
client = storage.Client()

# Define GCS bucket and zip file path
bucket_name = 'cris-voter-data'
zip_file_path = 'voter_activity_20220828.zip'
destination_path = 'temp.zip'

try:
    # Get the bucket
    bucket = client.get_bucket(bucket_name)
    print("Bucket:", bucket)

    # Get the blob (file) from the bucket
    blob = bucket.blob(zip_file_path)
    print("Blob:", blob)

    # Define the destination path for the downloaded file
    destination_path = 'temp.zip'

    # Download the blob's content and write it to a file
    with open(destination_path, 'wb') as file:
        blob.download_to_file(file)
    
    print("Downloaded zip file to:", destination_path)

    # Check if download was successful
    if os.path.exists(destination_path):
        # Extract the zip file
        with zipfile.ZipFile(destination_path, 'r') as zip_ref:
            zip_ref.extractall('extracted_files')
        print("Successfully extracted zip file contents.")
    else:
        print("Failed to download zip file.")

except Exception as e:
    print("Error:", e)


Bucket: <Bucket: cris-voter-data>
Blob: <Blob: cris-voter-data, voter_activity_20220828.zip, None>
Downloaded zip file to: temp.zip
Successfully extracted zip file contents.


### Since the data came without columns, I had to define column names, as well as schema

In [12]:
column_names = ['election_date', 'election_type', 'voter_id', \
                'tmp1', 'tmp2', 'tmp3', 'tmp4', 'tmp5', 'tmp6', 'tmp7', 'tmp8', \
                'zip', 'city', 'party_affiliation', \
                'tmp9', 'tmp10', \
                'ward', 'precinct', 'voter_status', \
                'tmp11', 'tmp12', 'tmp13', 'tmp14', 'tmp15', 'tmp16']

schema = {
  'election_date': 'object',
  'election_type': 'object',
  'voter_id': 'object',
  'tmp1': 'object',
  'tmp2': 'object',
  'tmp3': 'object',
  'tmp4': 'object',
  'tmp5': 'object',
  'tmp6': 'object',
  'tmp7': 'object',
  'tmp8': 'object',
  'zip': 'object',
  'city': 'object',
  'party_affiliation': 'object',
  'tmp9': 'object',
  'tmp10 ': 'object',
  'ward': 'object',
  'precinct': 'object',
  'voter_status': 'object',
  'tmp11': 'object',
  'tmp12': 'object',
  'tmp13': 'object',
  'tmp14': 'object',
  'tmp15': 'object',
  'tmp16': 'object'
}

### Here I'm reading one file and checking the data.

In [5]:
df = pd.read_csv('extracted_files/001_voter_act.txt', delimiter='|', names=column_names, dtype=schema)
df['election_date'] = pd.to_datetime(df['election_date'])
df['election_date'] = df['election_date'].dt.date

In [6]:
print('n_rows = ', len(df))

n_rows =  237190


In [7]:
earliest_date = df['election_date'].min()

print("Earliest date:", earliest_date)

Earliest date: 1995-08-22


### Now I define a loop from start number to end number.
I read the file and save it with selected columns, write it as parquet, and upload it to my Google cloud bucket.

In [10]:
from google.cloud import storage

client = storage.Client()

bucket = client.get_bucket('cris-voter-data')

selected_columns = ['election_date', 'election_type', 'voter_id', 'zip', 'city', 'party_affiliation', 'ward', 'precinct', 'voter_status']

def text_to_parquet(start, end):
    for n in range(start, end):
        filename = f"{n:03d}_voter_act.txt";
        df = pd.read_csv('extracted_files/' + filename, delimiter='|', names=column_names, dtype=schema)
        new_df = df[selected_columns]
        new_df.to_parquet(filename[:-4] + '.parquet', engine='pyarrow')
        blob = bucket.blob('output/' + filename[:-4] + '.parquet')
        blob.upload_from_filename(filename[:-4] + '.parquet')
        print({filename}, len(new_df))

In [13]:
text_to_parquet(340, 352)

{'340_voter_act.txt'} 53510
{'341_voter_act.txt'} 116536
{'342_voter_act.txt'} 311957
{'343_voter_act.txt'} 112542
{'344_voter_act.txt'} 412585
{'345_voter_act.txt'} 13006
{'346_voter_act.txt'} 312485
{'347_voter_act.txt'} 506841
{'348_voter_act.txt'} 1441063
{'349_voter_act.txt'} 3513
{'350_voter_act.txt'} 195544
{'351_voter_act.txt'} 410055


### These last cells allow me to read csv files that I created by hand.
Several files had bad data in them and could not be read by pandas. I downloaded these files to my computer and opened them in Excel, saved them as .csv files, and uploaded them back to the virtual machine. Then I could proceed.

In [2]:
import pandas as pd
from google.cloud import storage

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/cris/.gc/keys.json'

google_application_credentials = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')
print(google_application_credentials)

from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket('cris-voter-data')

selected_columns = ['election_date', 'election_type', 'voter_id', 'zip', 'city', 'party_affiliation', 'ward', 'precinct', 'voter_status']

/home/cris/.gc/keys.json


In [14]:
def csv_to_parquet(filename):
    df = pd.read_csv('extracted_files/' + filename, delimiter=',', names=column_names, dtype=schema)
    new_df = df[selected_columns]
    new_df.to_parquet(filename[:-4] + '.parquet', engine='pyarrow')
    blob = bucket.blob('output/' + filename[:-4] + '.parquet')
    blob.upload_from_filename(filename[:-4] + '.parquet')
    print(filename, len(new_df))

In [17]:
import glob

directory = 'extracted_files'

csv_files = glob.glob(os.path.join(directory, '*.csv'))

for file_path in csv_files:
    csv_to_parquet(os.path.basename(file_path))


177_voter_act.csv 136290
035c_voter_act.csv 729102
049b_voter_act.csv 524179
293_voter_act.csv 551527
137_voter_act.csv 439518
279_voter_act.csv 111978
291_voter_act.csv 266369
338_voter_act.csv 193082
035f_voter_act.csv 709986
035d_voter_act.csv 732746
258_voter_act.csv 528017
072_voter_act.csv 384265
035h_voter_act.csv 809793
274_voter_act.csv 915640
229_voter_act.csv 660342
035b_voter_act.csv 732359
176_voter_act.csv 741171
246_voter_act.csv 439868
035g_voter_act.csv 715503
289_voter_act.csv 47202
035e_voter_act.csv 730499
165_voter_act.csv 540971
049a_voter_act.csv 599999
163_voter_act.csv 752367
178_voter_act.csv 458873
035a_voter_act.csv 732422


### Clean up

In [18]:
!rm *.parquet